In [1]:
!nvidia-smi

Mon Jan 11 21:25:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.89       Driver Version: 460.89       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P0    26W /  N/A |   1276MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.device('cuda:12945')

(device(type='cpu'),
 device(type='cuda', index=12945))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):
    if torch.cuda.device_count() > i:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device = try_gpu())
X, X.device

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 device(type='cuda', index=0))

In [7]:
Y = torch.rand(2, 3, device = try_gpu(10))
Y, Y.device

(tensor([[0.1155, 0.3801, 0.1897],
         [0.5263, 0.2275, 0.5761]]),
 device(type='cpu'))

In [8]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.1155, 0.3801, 0.1897],
        [0.5263, 0.2275, 0.5761]])
tensor([[0.1155, 0.3801, 0.1897],
        [0.5263, 0.2275, 0.5761]], device='cuda:0')


In [9]:
Z+X

tensor([[1.1156, 1.3801, 1.1897],
        [1.5263, 1.2275, 1.5761]], device='cuda:0')

In [10]:
Z.cuda(0) is Z

True

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [12]:
net(X)

tensor([[-0.2322],
        [-0.2322]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

In [14]:
X = torch.rand(1000,1000)
Y = torch.rand(1000, 1000)

In [15]:
%timeit torch.matmul(X, Y)

6.92 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
X = torch.rand(1000, 1000, device = torch.device('cuda:0'))
Y = torch.rand(1000, 1000, device = torch.device('cuda:0'))

In [17]:
%timeit torch.matmul(X, Y)

576 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
